# Ensemble Model 
Classification by 3 experts

## Prepare Input Data

In [2]:
import sys
sys.path.append("..")
import numpy as np
import sklearn as sk
from sklearn.metrics import classification_report
import torch
from torch import nn, cuda
import matplotlib.pyplot as plt
import prepare_data
from prepare_data import DataPreparationContext
from prepare_data import ClassEmbedders
import models
from models import GenericMutliClassifier

In [3]:
data_context = DataPreparationContext(combine_riskiest=16, verbose=True)

validation_data = data_context.prepare_from_file("../../symptom_sum_top16/test.pkl", oversample=False)
X_val_np = validation_data['X']
y_val_np = validation_data['y']

100%|██████████| 2643/2643 [00:03<00:00, 742.88it/s]

Input File: ../../symptom_sum_top16/test.pkl
 [depression] has 62
 [depression anxiety] has 44
 [anxiety] has 21
 [ptsd] has 7
 [depression anxiety ptsd] has 3
 [anxiety ptsd] has 7
 [depression ptsd] has 2
 [] has 2054



In [4]:
X_val = models.convert_to_tensors(X_val_np).type(torch.float32).to(models.device)
y_val = models.convert_to_tensors(y_val_np).type(torch.LongTensor).to(models.device)

## 1. Depression Expert

In [5]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("depression-expert.pth")
has_depression = model.evaluate_model(X_val).cpu()

## 2. Anxiety Expert

In [6]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("anxiety-expert.pth")
has_anxiety = model.evaluate_model(X_val).cpu()

## 3. PTSD Expert

In [7]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("ptsd-expert.pth")
has_ptsd = model.evaluate_model(X_val).cpu()

## Evaluate Results

In [11]:
val_pred = []
val_true = []
y_val_cpu = y_val.cpu()
labels =  [' '.join(x) for x in prepare_data.CLASSES_COMORBID.values()]

for i in range(len(X_val_np)):
    diseases = []
    if has_depression[i] == 2:
        diseases.append("depression")
    if has_anxiety[i] == 2:
        diseases.append("anxiety")
    if has_ptsd[i] == 2:
        diseases.append("ptsd")
    diseases = frozenset(diseases)
    disease_class = ClassEmbedders.match_exact(diseases, data_context.classes, data_context.classes_lookup)
    val_pred.append(disease_class)
    val_true.append(y_val_cpu[i].numpy())
        
print(classification_report(y_true=val_true, y_pred=val_pred, target_names=labels, labels=[4,5,6,7]))

                         precision    recall  f1-score   support

     depression anxiety       0.15      0.27      0.19        44
        depression ptsd       0.00      0.00      0.00         2
           anxiety ptsd       0.00      0.00      0.00         7
depression anxiety ptsd       0.20      0.33      0.25         3

              micro avg       0.13      0.23      0.17        56
              macro avg       0.09      0.15      0.11        56
           weighted avg       0.13      0.23      0.16        56

